# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 0.5346791174347928),
 (1, 0.5346791174347928),
 (2, 0.5346791174347928),
 (3, 0.5346791174347928),
 (4, 0.5346791174347928)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('what', 10)]),
 (1, [('is', 18), ('it', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.007606532972857183),
   (None, 0.03456635900700644),
   (None, 0.09277873387431923),
   (None, 0.11127025319412676),
   (None, 0.16757955297109495),
   (None, 0.23988933750322006),
   (None, 0.26375871285147734),
   (None, 0.2656554311466679),
   (None, 0.28165971088749664),
   (None, 0.30443391254575225),
   (None, 0.4215059910754553),
   (None, 0.4539334105682241)]),
 (1,
  [(None, 0.51965224074568),
   (None, 0.530714566247857),
   (None, 0.5662119158040747),
   (None, 0.6270392623282736),
   (None, 0.6373510531715275),
   (None, 0.7380178256060345),
   (None, 0.7569096007633126),
   (None, 0.7622858539700159),
   (None, 0.7970742444526275),
   (None, 0.7986763292924883),
   (None, 0.8297146171793601),
   (None, 0.8356558236572637),
   (None, 0.8490954921328405),
   (None, 0.8758021758947807),
   (None, 0.886178300571316),
   (None, 0.9665361250040141),
   (None, 0.985253801292454),
   (None, 0.998838667522952)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random
from typing import List

def RECORDREADER():
    return [(index, num) for (index, num) in enumerate(np.random.randint(150, size=500))]

def MAP(key: int, value: int):
    yield (1, value)

def REDUCE(key: int, row: List[int]):
    yield max(row)

output = list(MapReduce(RECORDREADER, MAP, REDUCE))
output

[149]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def REDUCE(key: int, row: List[int]):
    yield sum(row) / len(row)

output = list(MapReduce(RECORDREADER, MAP, REDUCE))
output

[75.226]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def GroupByKey(iterable):
  t = {}
  for (key1, val1) in iterable:
    t[key1] = sorted(t.get(key1, []) + [val1])

  return t.items()

print(GroupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))
print(groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))

dict_items([(1, [0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 7, 8, 8, 8, 8, 8, 9, 10, 10, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 16, 16, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 25, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 32, 33, 33, 33, 33, 34, 34, 34, 35, 36, 37, 37, 37, 38, 38, 38, 38, 39, 40, 40, 40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 47, 47, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 52, 52, 52, 52, 53, 53, 54, 54, 54, 54, 54, 55, 55, 56, 56, 57, 57, 57, 57, 57, 57, 57, 58, 59, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 61, 62, 62, 62, 62, 62, 63, 63, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 67, 68, 68, 68, 69, 70, 70, 70, 71, 71, 72, 72, 72, 73, 73, 73, 73, 73, 74, 74, 74, 75, 75, 75, 76, 76, 76, 77, 77, 77, 77, 78, 78, 78, 78,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
def RECORDREADER():
    return [(index, value) for (index, value) in enumerate(np.random.randint(10, size=500))]
def MAP(key: int, value: int):
    yield (value, value)
def REDUCE(key: int, rows: List[int]):
    yield key

output = list(MapReduce(RECORDREADER, MAP, REDUCE))
output

[9, 6, 5, 8, 0, 4, 2, 1, 3, 7]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [27]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 1


def RECORDREADER():
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(250)]

record = RECORDREADER()
part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[dict_items([((91, 8, 76), [(91, 8, 76)]), ((75, 2, 58), [(75, 2, 58)]), ((91, 0, 58), [(91, 0, 58)]), ((89, 7, 5), [(89, 7, 5)]), ((81, 78, 81), [(81, 78, 81)])]), dict_items([((75, 14, 58), [(75, 14, 58)]), ((51, 46, 91), [(51, 46, 91)])]), dict_items([((3, 5, 65), [(3, 5, 65)]), ((99, 100, 62), [(99, 100, 62)])]), dict_items([((69, 44, 65), [(69, 44, 65)]), ((67, 17, 56), [(67, 17, 56)]), ((31, 40, 19), [(31, 40, 19)]), ((35, 21, 0), [(35, 21, 0)])]), dict_items([((87, 16, 59), [(87, 16, 59)]), ((33, 64, 4), [(33, 64, 4)]), ((45, 91, 94), [(45, 91, 94)]), ((99, 52, 32), [(99, 52, 32)])]), dict_items([((29, 21, 98), [(29, 21, 98)])]), dict_items([((7, 15, 41), [(7, 15, 41)]), ((37, 15, 27), [(37, 15, 27)]), ((81, 7, 3), [(81, 7, 3)])]), dict_items([((93, 38, 44), [(93, 38, 44)]), ((3, 31, 53), [(3, 31, 53)]), ((53, 18, 69), [(53, 18, 69)]), ((55, 32, 72), [(55, 32, 72)])]), dict_items([((57, 32, 98), [(57, 32, 98)]), ((31, 54, 63), [(31, 54, 63)])]), dict_items([((99, 17, 11), [(99, 

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [29]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def GroupByKey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))

shuffle_output = GroupByKey(map_output)
shuffle_output = list(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(), (12,), (1,)]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [32]:
def RECORDREADER(n, count):
    return [(random.randint(0, n), random.randint(0, n)) for i in range(count)]
def MAP(t): return (t, t)
def REDUCE(key, values: Iterator[NamedTuple]): return (key, key)

record = RECORDREADER(100, 100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100,100)))

shuffle_output = GroupByKey(map_output)
shuffle_output = list(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(89, 26), (68, 0), (26, 98), (93, 58), (76, 54), (69, 43), (15, 57), (2, 25), (25, 70), (29, 56), (1, 32), (66, 76), (98, 21), (54, 85), (87, 27), (32, 72), (62, 28), (48, 46), (34, 30), (75, 64), (88, 37), (34, 53), (4, 4), (100, 17), (86, 36), (11, 81), (16, 43), (2, 22), (48, 5), (21, 61), (14, 34), (69, 16), (3, 44), (35, 10), (23, 34), (23, 30), (59, 70), (17, 8), (38, 59), (64, 2), (95, 33), (4, 1), (43, 27), (25, 3), (71, 76), (62, 85), (67, 3), (42, 23), (6, 4), (20, 14), (31, 0), (33, 84), (90, 82), (97, 75), (92, 97), (68, 17), (14, 16), (43, 25), (41, 4), (100, 9), (1, 86), (19, 59), (4, 6), (93, 63), (18, 84), (95, 64), (100, 69), (80, 37), (21, 92), (48, 6), (84, 8), (42, 13), (97, 58), (79, 47), (5, 79), (48, 63), (97, 28), (75, 60), (73, 68), (89, 24), (40, 33), (96, 68), (69, 24), (83, 17), (24, 61), (76, 9), (10, 93), (61, 25), (79, 24), (74, 6), (96, 55), (40, 32), (99, 65), (14, 76), (73, 79), (31, 12), (43, 64), (45, 22), (32, 83), (46, 56)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [36]:
def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

record = RECORDREADER(5, 100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(5,100)))
print(map_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), list(GroupByKey(map_output)))) if el is not None]
print(reduce_output)

[(1, 4), (0, 3), (2, 2), (0, 0), (4, 3), (2, 1), (5, 5), (1, 5), (1, 1), (4, 0), (3, 3), (5, 4), (2, 0), (5, 3), (3, 2), (4, 3), (0, 0), (0, 1), (5, 0), (1, 0), (4, 2), (2, 0), (1, 3), (2, 0), (5, 2), (2, 3), (2, 1), (5, 0), (0, 5), (2, 2), (5, 2), (4, 0), (0, 4), (0, 2), (2, 0), (4, 1), (0, 3), (4, 3), (2, 3), (2, 1), (3, 3), (0, 0), (2, 1), (2, 1), (5, 0), (5, 5), (4, 2), (4, 5), (0, 2), (4, 5), (0, 2), (1, 1), (2, 4), (1, 1), (2, 3), (0, 4), (0, 4), (2, 2), (3, 2), (0, 5), (3, 0), (4, 1), (2, 5), (1, 3), (3, 2), (4, 1), (0, 5), (3, 3), (0, 3), (4, 3), (1, 4), (4, 2), (1, 2), (3, 0), (3, 4), (0, 5), (0, 0), (4, 0), (3, 5), (2, 2), (4, 4), (0, 5), (0, 2), (0, 2), (5, 3), (1, 4), (2, 2), (1, 0), (5, 2), (0, 1), (0, 3), (4, 3), (5, 2), (5, 2), (4, 3), (5, 0), (4, 3), (2, 2), (2, 2), (1, 3)]
[((5, 1), (5, 1)), ((1, 4), (1, 4)), ((2, 2), (2, 2)), ((2, 5), (2, 5)), ((2, 5), (2, 5)), ((2, 0), (2, 0)), ((4, 5), (4, 5)), ((4, 2), (4, 2)), ((1, 4), (1, 4)), ((2, 4), (2, 4)), ((1, 3), (1, 3)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [40]:
rels = [1, 2]
class Tuple:
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(5) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]:
    return (key, key)

record = RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), list(GroupByKey(map_output)))) if el is not None]
print(reduce_output)

[(((2, 1), (1, 0), (1, 0), (2, 2), (0, 0)), 2), (((3, 3), (1, 3), (3, 3), (1, 1), (1, 0)), 1), (((3, 2), (0, 3), (2, 1), (2, 1), (2, 3)), 1), (((2, 0), (3, 2), (1, 3), (3, 3), (1, 1)), 2), (((1, 2), (3, 0), (0, 0), (1, 3), (3, 0)), 2), (((3, 2), (2, 0), (2, 0), (0, 2), (1, 2)), 2), (((3, 1), (3, 1), (1, 0), (1, 2), (1, 1)), 1), (((3, 3), (2, 2), (3, 1), (3, 2), (3, 2)), 1), (((1, 3), (2, 3), (0, 1), (2, 2), (0, 1)), 2), (((3, 2), (3, 3), (3, 2), (1, 1), (2, 2)), 1), (((1, 0), (0, 3), (2, 1), (3, 2), (3, 1)), 1), (((0, 2), (3, 0), (0, 3), (0, 2), (2, 0)), 1), (((2, 0), (1, 0), (3, 0), (2, 2), (3, 2)), 2), (((0, 0), (1, 3), (2, 2), (3, 1), (3, 3)), 1), (((3, 1), (2, 3), (0, 0), (2, 2), (0, 0)), 2), (((2, 1), (2, 0), (3, 1), (3, 3), (0, 2)), 1), (((2, 3), (3, 1), (3, 1), (0, 0), (0, 1)), 1), (((1, 3), (3, 3), (2, 0), (0, 3), (3, 1)), 1), (((3, 2), (1, 0), (0, 0), (3, 1), (3, 3)), 1), (((0, 0), (1, 2), (0, 2), (2, 1), (3, 2)), 2), (((3, 3), (3, 0), (2, 3), (2, 2), (0, 2)), 1), (((1, 1), (0

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [42]:
def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

record = RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)
reduce_output = list(map(lambda x: REDUCE(*x), list(GroupByKey(map_output))))
print(reduce_output)

[(0, (2, 3)), (1, (2, 0)), (2, (1, 0)), (1, (2, 2)), (1, (2, 0)), (0, (2, 0)), (1, (2, 1)), (1, (2, 1)), (0, (1, 3)), (2, (1, 1)), (3, (1, 3)), (3, (2, 3)), (2, (1, 3)), (0, (1, 2)), (0, (2, 1)), (0, (1, 3)), (2, (2, 0)), (0, (2, 3)), (1, (2, 0)), (3, (1, 2)), (1, (1, 0)), (3, (1, 1)), (0, (1, 3)), (3, (2, 3)), (1, (1, 1)), (0, (1, 3)), (3, (2, 0)), (3, (2, 2)), (0, (1, 0)), (2, (2, 1)), (0, (1, 3)), (1, (2, 2)), (3, (1, 2)), (3, (1, 2)), (3, (2, 3)), (1, (2, 2)), (1, (2, 1)), (0, (2, 0)), (1, (2, 0)), (2, (1, 2)), (2, (2, 2)), (2, (1, 0)), (1, (1, 1)), (2, (2, 2)), (3, (1, 1)), (3, (1, 3)), (1, (2, 3)), (1, (2, 1)), (2, (1, 1)), (0, (2, 2)), (1, (1, 1)), (2, (1, 2)), (3, (1, 1)), (1, (2, 0)), (0, (1, 1)), (1, (2, 1)), (2, (2, 0)), (3, (2, 0)), (0, (1, 3)), (3, (2, 0)), (1, (2, 1)), (0, (1, 2)), (3, (1, 1)), (0, (2, 3)), (2, (2, 2)), (3, (1, 2)), (3, (1, 1)), (0, (1, 2)), (0, (2, 1)), (2, (1, 0)), (1, (1, 0)), (1, (1, 1)), (0, (2, 2)), (3, (1, 3)), (3, (2, 1)), (0, (2, 3)), (1, (2, 0))

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [43]:
def get_random_tuple():
  return (random.randint(0, 5), random.randint(0, 5), random.randint(0, 5))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def theta(values):
  return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):
  x = theta(values)
  return (key, x)

record = RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)
reduce_output = list(map(lambda x: REDUCE(*x), list(GroupByKey(map_output))))
print(reduce_output)

[(2, 4), (5, 2), (5, 3), (2, 0), (3, 2), (3, 3), (0, 1), (0, 5), (2, 4), (4, 2), (0, 0), (0, 2), (4, 0), (5, 3), (3, 4), (3, 2), (4, 1), (5, 3), (1, 2), (5, 4), (2, 3), (4, 2), (5, 0), (5, 4), (2, 0), (1, 1), (5, 2), (5, 3), (1, 0), (0, 4), (5, 2), (2, 3), (4, 2), (5, 1), (0, 3), (3, 0), (0, 1), (4, 5), (3, 2), (1, 4), (5, 1), (1, 5), (3, 1), (4, 5), (4, 1), (1, 3), (2, 3), (0, 3), (1, 5), (2, 0), (0, 5), (2, 4), (1, 2), (4, 2), (0, 0), (5, 5), (4, 2), (1, 0), (2, 1), (5, 5), (2, 1), (4, 0), (5, 0), (0, 1), (1, 2), (1, 2), (0, 5), (1, 2), (3, 1), (0, 1), (2, 2), (4, 1), (5, 3), (4, 5), (4, 1), (2, 3), (5, 3), (0, 1), (3, 1), (4, 4), (2, 0), (3, 3), (0, 5), (5, 4), (3, 4), (2, 5), (2, 2), (4, 3), (5, 3), (2, 4), (3, 5), (5, 3), (4, 0), (3, 1), (3, 4), (5, 1), (2, 5), (0, 2), (3, 2), (5, 3)]
[(2, 44), (5, 58), (3, 35), (0, 39), (4, 36), (1, 28)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [44]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [45]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range (I):
    yield ((i, k), w * small_mat[i][j])


def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

Проверьте своё решение

In [46]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [47]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [49]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      yield ((0, i, j), small_mat[i, j])

  for j in range(J):
    for k in range(K):
      yield ((1, j, k), big_mat[j, k])

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1
  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))


def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)


def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_RECORDREADER(iterable):
  def func():
    for el in iterable:
      yield el
  return func


reference_solution = np.matmul(small_mat, big_mat)
partial_solution = MapReduce(RECORDREADER, MAP, REDUCE)
solution = MapReduce(GET_RECORDREADER(partial_solution), MAP_IDENTITY, REDUCE_AGREGATE)

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [52]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i, j), matrix[i, j])

  yield RECORDREADER(0, small_mat)
  yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [53]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix, shift):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i + shift, j), matrix[i, j])

  split_size = int(np.ceil(len(small_mat) / maps))

  for i in range(0, len(small_mat), split_size):
    yield RECORDREADER(0, small_mat[i:i + split_size], i)

  split_size = int(np.ceil(len(big_mat) / maps))

  for i in range(0, len(big_mat), split_size):
    yield RECORDREADER(1, big_mat[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_index, row_index, col_index) = k1

  if matrix_index == 0:
    yield (col_index, (matrix_index, row_index, v1))
  else:
    yield (row_index, (matrix_index, col_index, v1))

def REDUCE(key, values):
  first_matrix = [el for el in values if el[0] == 0]
  second_matrix = [el for el in values if el[0] == 1]

  for first_el in first_matrix:
    for second_el in second_matrix:
      (_, first_index, first_value) = first_el
      (_, second_index, second_value) = second_el
      yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(*args):
  yield args

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

solution = MapReduceDistributed(GET_INPUTFORMAT(partitioned_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
solution = [(partition_id, list(partition)) for (partition_id, partition) in solution]

solution_unpacked = []
for value in solution:
  for v in value[1]:
    solution_unpacked.append(v)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i, k), vw) in reduce_output) + 1
  K = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution_unpacked)) # should return true

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True